In [2]:
!pip install ultralytics

     -------------------------------------- 616.7/616.7 kB 7.8 MB/s eta 0:00:00
  Using cached scipy-1.7.3-cp37-cp37m-win_amd64.whl (34.1 MB)
     ------------------------------------- 614.9/614.9 kB 12.9 MB/s eta 0:00:00
     ------------------------------------- 614.4/614.4 kB 12.9 MB/s eta 0:00:00
     ------------------------------------- 606.9/606.9 kB 19.2 MB/s eta 0:00:00
     ------------------------------------- 606.2/606.2 kB 18.6 MB/s eta 0:00:00
     ------------------------------------- 605.5/605.5 kB 19.2 MB/s eta 0:00:00
     ------------------------------------- 605.6/605.6 kB 19.2 MB/s eta 0:00:00
  Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)


In [3]:
from ultralytics import YOLO
import os
from PIL import Image
import numpy as np

d:\miniconda\envs\machine_learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = YOLO(r"C:\Users\internece\Downloads\yolov8m.pt")

In [6]:
destination_path = r"C:\Users\internece\Remote-PD-Detection\segmented_yolov8"
frame_path = r"C:\Users\internece\Remote-PD-Detection\frames"

In [7]:
white_bg = Image.open(r"C:\Users\internece\Remote-PD-Detection\white.png")
white_bg = np.array(white_bg)

In [ ]:
os.makedirs(destination_path, exist_ok=True)
for severity in os.listdir(frame_path):
    if severity == "NORMAL" or severity == "SEVERE":
        os.makedirs(os.path.join(destination_path, severity), exist_ok=True)
        for video in os.listdir(os.path.join(frame_path, severity)):
            os.makedirs(os.path.join(destination_path, severity, video), exist_ok=True)
            for image in os.listdir(os.path.join(frame_path, severity, video)):
                results = model(os.path.join(frame_path, severity, video, image), save=True, show_labels=True, show_conf=True, line_width=0)
                for r in results:
                    im_array = r.plot(img=white_bg, labels=False, boxes=False)
                    im = Image.fromarray(im_array)  
                    im.save(os.path.join(destination_path, severity, video, image))

d:\miniconda\envs\machine_learning\lib\site-packages\torch\cuda\__init__.py:132: UserWarning: 
    Found GPU0 NVIDIA GeForce GT 730 which is of cuda capability 3.5.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 3.7.
    
  warnings.warn(old_gpu_warn % (d, name, major, minor, min_arch // 10, min_arch % 10))

